In [1]:
#加载数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#一层全连接网络的
import tensorflow as tf

sess = tf.Session()

x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

#定义参数
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

#定义预测值
y = tf.matmul(x,W) + b

#定义损失函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits =y))


#求最佳
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

#开始训练
for i in range(1000):
    x_batch,y_batch = mnist.train.next_batch(100) #每次取100个
    sess.run(train_step,feed_dict = {x:x_batch,y_true:y_batch})
    if i % 100 == 0:
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_true,1))#产生一个长度为测试例子个数的bool数组，预测对了就是true
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #tf.cast把bool类型转换成float32
        print(str(i)+":   "+ str(sess.run(accuracy,{x:mnist.test.images,y_true:mnist.test.labels})))
        
print("#############################")
print("final:   "+ str(sess.run(accuracy,{x:mnist.test.images,y_true:mnist.test.labels})))


0:   0.4075
100:   0.8948
200:   0.9031
300:   0.9074
400:   0.9037
500:   0.9125
600:   0.9163
700:   0.9157
800:   0.9158
900:   0.9174
#############################
final:   0.9187


In [3]:
#使用卷积网络的方法

#定义权重/卷积核
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) #生成截断的自然分布,stddev是标准差
    return tf.Variable(initial)

#定义fc中的偏移量
def bais_variable(shape):
    initial = tf.constant(0.1, shape=shape)#这里偏移量用的是固定值
    return tf.Variable(initial)

#定义卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME") 
#这里x是输入的数据，W为卷积核，必须strides[0]=strides[3]=1,strides[1]是height的步长，strides[2]是width的步长。
#padding 可以选两个参数，VALID是没有补，SAME是补0   http://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t
#x、W为四维张量，x[batch, in_height, in_width, in_channels] ,W[filter_height, filter_width, in_channels, out_channels]


#定义最大池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")#这里的ksize与Strides类似

#定义第一个卷基层的卷积核与偏移量
W_conv1 = weight_variable([5,5,1,32]) #5x5x1的卷积核，输出32个通道
b_conv1 = bais_variable([32])

#变换原始数据
x_image = tf.reshape(x,[-1,28,28,1]) #第一个维度为-1，代表第一个维度可以不限制大小，换句话说，就是图片的batch

#第一层输出
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1) #通过relu激活函数
h_pool1 = max_pool_2x2(h_conv1)


#写第二个卷基层
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bais_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#输入到fc中
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bais_variable([1024])

#第二个池化出来的数据降维
h_pool2_plat = tf.reshape(h_pool2,[-1,7*7*64])

#计算第一个fc层
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_plat,W_fc1) + b_fc1)

#添加dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob) #keep_drop : The probability that each element is kept.


#第二个fc层
W_fc2 = weight_variable([1024,10])
b_fc2 = bais_variable([10])


#最后结果
y_conv = tf.matmul(h_fc1_drop,W_fc2) + b_fc2


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_true,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

sess.run(tf.global_variables_initializer())

for i in range(20000):
    x_batch,y_batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        print(str(i)+":  "+ str(sess.run(accuracy,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,keep_prob:1})))
    sess.run(train_step,feed_dict={x:x_batch,y_true:y_batch,keep_prob:0.5})
    #dropout在训练的时候随机失活，在预测的时候便不再失活。
    
    
print("#####################")
print("final:   " + str(sess.run(accuracy,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,keep_prob:1})))

0:  0.1064
100:  0.8286
200:  0.9054
300:  0.927
400:  0.941
500:  0.9437
600:  0.95
700:  0.9548
800:  0.9563
900:  0.9599
1000:  0.963
1100:  0.9643
1200:  0.9684
1300:  0.9678
1400:  0.9694
1500:  0.9701
1600:  0.9738
1700:  0.9736
1800:  0.9743
1900:  0.975
2000:  0.9765
2100:  0.9751
2200:  0.9755
2300:  0.9777
2400:  0.9791
2500:  0.9789
2600:  0.9787
2700:  0.9791
2800:  0.9811
2900:  0.9802
3000:  0.9802
3100:  0.9812
3200:  0.9789
3300:  0.9832
3400:  0.9808
3500:  0.9824
3600:  0.9815
3700:  0.9812
3800:  0.9829
3900:  0.9833
4000:  0.9832
4100:  0.9852
4200:  0.9854
4300:  0.984
4400:  0.9845
4500:  0.9853
4600:  0.9825
4700:  0.9858
4800:  0.9855
4900:  0.9849
5000:  0.986
5100:  0.9855
5200:  0.9845
5300:  0.9875
5400:  0.9855
5500:  0.9875
5600:  0.9864
5700:  0.9875
5800:  0.9876
5900:  0.9871
6000:  0.9872
6100:  0.9889
6200:  0.9873
6300:  0.9877
6400:  0.9865
6500:  0.9875
6600:  0.9882
6700:  0.9894
6800:  0.9889
6900:  0.9896
7000:  0.9893
7100:  0.9887
7200:  0.988

In [ ]:
#这个是我按照tensorflow（https://www.tensorflow.org/get_started/mnist/pros）这个教程上面写的。并且把每一行都看懂，并写下注释，可能对我自己有用